In [1]:
import torch
from tqdm import tqdm
from transformers import (
    AutoConfig,
    AutoModelForMultipleChoice,
    AutoTokenizer,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
import numpy as np
# np.argmax([1,2,3,4])

In [2]:
def softmax(x, axis=None):
    x = np.array(x)
    x = x - x.max(axis=axis, keepdims=True)
    y = np.exp(x)
    rst = y / y.sum(axis=axis, keepdims=True)
    return rst.tolist()
softmax([1,2])

[0.2689414213699951, 0.7310585786300049]

In [3]:
tokenizer = AutoTokenizer.from_pretrained('ethanyt/guwenbert-base')

In [4]:
import json
with open('data/valid.jsonl','r') as f:
    valid_json = f.readlines()
    valid_json = [json.loads(e) for e in valid_json]

In [5]:
import pickle
with open('/home/zhangkechi/workspace/nlp_course/baseline/CCPM-baseline-main/predict/results_guwen/test_result.pkl','rb') as f:
    rst = pickle.load(f)

In [26]:
rst['test_result'].predictions[0].tolist()

[-0.09412198513746262, 8.516458511352539, 9.14784049987793, 12.8546142578125]

In [19]:
rst['test_dataset'],rst['test_dataset'][0].keys()

(Dataset({
     features: ['answer', 'attention_mask', 'choices', 'input_ids', 'labels', 'token_type_ids', 'translation'],
     num_rows: 2720
 }),
 dict_keys(['answer', 'attention_mask', 'choices', 'input_ids', 'labels', 'token_type_ids', 'translation']))

In [23]:
tokenizer.decode(rst['test_dataset'][0]['input_ids'])

'[CLS] 昏 暗 的 灯 熄 灭 了 又 被 重 新 点 亮 。 [SEP] 灭 又 明 [SEP]'

In [27]:
all_scores = {}
for i in tqdm(range(len(rst['test_dataset']))):
    idx = i
    all_scores[idx] = softmax(rst['test_result'].predictions[i].tolist())

100%|█████████████████████████████████████████████████████████████████████████| 2720/2720 [00:00<00:00, 38844.21it/s]


In [38]:
valid_json[0]

{'translation': '昏暗的灯熄灭了又被重新点亮。',
 'choices': ['渔灯灭复明', '残灯灭又然', '残灯暗复明', '残灯灭又明'],
 'answer': 3}

3

In [29]:
predict_valid_json = []
for i in range(len(valid_json)):
    choices = valid_json[i]['choices']
    this_scores = all_scores[i]
    this_dict = {k:valid_json[i][k] for k in valid_json[i]}
    this_dict['scores'] = this_scores
    this_dict['predict'] = np.argmax(this_scores)
    this_dict['true'] = (this_dict['predict'] == this_dict['answer'])
    predict_valid_json.append(this_dict)

In [31]:
true_num = np.sum([int(e['true']) for e in predict_valid_json]) / len(predict_valid_json)
print(true_num)

0.8830882352941176


In [77]:
predict_valid_json[-10]

{'translation': '请莫嫌弃潇湘一带人烟稀少。',
 'choices': ['莫厌潇湘少人处', '飞时莫近潇湘浦', '莫厌皖山穷绝处', '潇湘云水人何处'],
 'answer': 0,
 'scores': [2.951438719346828,
  2.0997304880167835,
  1.9610124685063726,
  2.1316768167366544],
 'predict': 0,
 'true': True}

In [64]:
true_num

0.8253676470588235

In [4]:
rst.predictions.shape

(24797, 2)

PredictionOutput(predictions=array([[ 0.03430561, -0.48208067],
       [ 0.09326166, -0.5414783 ],
       [ 0.33604988, -0.7945821 ],
       ...,
       [ 1.0819488 , -1.4716136 ],
       [ 1.0384777 , -1.4446024 ],
       [ 1.1046293 , -1.4878067 ]], dtype=float32), label_ids=array([1, 0, 0, ..., 0, 0, 1]), metrics={'test_loss': 0.4463875889778137, 'test_accuracy': 0.8132838606834412, 'test_runtime': 28.3801, 'test_samples_per_second': 873.745, 'test_steps_per_second': 17.477})

In [80]:
sum(rst['test_result'].label_ids)/rst['test_result'].label_ids.shape[0]

0.30104448118724036

In [82]:
rst['test_result'].label_ids.shape

(24797,)

In [83]:
rst['test_result'].label_ids.sum()

7465